# Standardizing Mass Spectrometry Data to Calculate Periplasmic Protein Density

In [33]:
import numpy as np
import pandas as pd 
import size.viz # Custom module for specifying plotting colors
import altair as alt
import scipy.stats
import scipy.optimize
colors, palette = size.viz.altair_style()

This notebook covers my approach of exploring mass spectrometry data to see how periplasmic protein content and density scale with growth rate.

## Brief Overview of Datasets
Most mass spectrometry measurements are not in absolute units, meaning they are not in proper "per cell" measurements. I recently tried to assemble a list of datasets that are trustworthy in their reporting, but I quickly discovered that there's a pretty wide breadth in how people define "per cell" and the controls that are needed to ensure it is correct. In general, I only really believe relative abundance measurements. 

Luckily, we know quite a bit about basic aspects of *E. coli* physiology, meaning we know some aspects of the typical protein density, have empirical relations for how cell volume scales with growth rate, and have annotation of nearly the entire proteome at a functional and localization level. Using the relative abundance measurements and our knowledge of the molecular weight of individual proteins, we can pretty easily compute a mass fraction of the proteome occupied by any single protein. Using these aforementioned empirical measurements, we can calculate our way to "per cell" measurements from relative measurements in mass spectrometry. 

## The Procedure
I've collected a set of mass spectrometry measurements and have standardized their gene names such that they are directly comparable. The sources of these measurements are as follows. 


* Schmidt *et al.* "The Quantitative and Condition-Dependent *Escherichia coli* Proteome." Nat Biotechnol 2016, 34 (1), 104–110. https://doi.org/10.1038/nbt.3418.


* Valgepea, K.; Adamberg, K.; Seiman, A.; Vilu, R. *Escherichia coli* Achieves Faster Growth by Increasing Catalytic and Translation Rates of Proteins. Mol. BioSyst. 2013, 9 (9), 2344–2358. https://doi.org/10.1039/C3MB70119K.


* Peebo *et al.*. "Proteome Reallocation in Escherichia coli with Increasing Specific Growth Rate". Molecular BioSystems 2015, 11 (4), 1184–1193. https://doi.org/10.1039/C4MB00721B.


* Mori *et al.* "From Coarse to Fine: The Absolute *Escherichia coli* Proteome under Diverse Growth Conditions." Mol Syst Biol 2021, 17 (5). https://doi.org/10.15252/msb.20209536. **Note that I only used some of the data from this work as I wanted to avoid N- and R-limitation as well as osmotic stress**


* Caglar *et al.*. "The *E. coli* Molecular Phenotype under Different Growth Conditions." Sci Rep 2017, 7 (1), 1–15. https://doi.org/10.1038/srep45303 **I only considered the C-limitation conditions**


* Soufi *et al.*  "Characterization of the *E. coli* Proteome and Its Modifications during Growth and Ethanol Stress." Frontiers in Microbiology 2015, 6. https://doi.org/10.3389/fmicb.2015.00103. **I did not use the Ethanol stress samples.**


People are generally terrible at providing their data in an actually useable form, so it took a lot of work to get this into a standardized format. I've saved these data in a single, long-form tidy dataframe in this repository (`/data/source/compiled_mass_fractions.csv`). 

To get to a standardized "per cell" resolution, we have to know details of how much protein there is per cell. This can be a bit difficult to directly measure with any appreciable level of certainty, so one approach is to use our empirical knowledge of the total *protein density* of a cell and how the cell *volume* changes with growth rate. For the former, Basan et al. 2015 did a careful measurement of total protein per OD and cells per OD as a function of growth rate. They also measured the cell volume, though that seems to be systematically over estimated. Si et al 2017 and 2019 did very careful measurements of cell dimensions and total cell volume across growth rates and for multiple strain backgrounds. We can use both of these data sets to figure out what the protein density is per cell.

## Empirical Fitting for Cell Length, Width, and Volume
We will start with figuring out some empirical relations to describe how the total cell volume changes as a function of growth rate for 
two strain background -- NCM3722 and MG1655. To begin, we will load and view the data.

In [30]:
# Load the cell volume data
vol_data = pd.read_csv('../../data/source/Si2017/Si2017_c_limitation.csv')

# Set up the plotting base
vol_plot = alt.Chart(
                data=vol_data,
                width=300,
                height=300,
                ).mark_point(
                    size=100,
                    opacity=0.75
                ).encode(
                    x=alt.X('growth_rate_hr:Q'),
                    y=alt.Y(alt.repeat("column"), type='quantitative'),
                    color=alt.Color('strain:N')
                ).repeat(
                    column=['cell_length_um', 'cell_width_um', 'cell_volume_fL']
                ).properties(
                    title='Si et al., 2017 - cell dimension measurements'
                )
vol_plot

alt.RepeatChart(...)

There are clear differences between MG1655 and NCM3722, which was kind of expected. As *E. coli* cells grow exponentially, we can do an empirical fit of an exponential with the form
$$
V_{cell} = ae^{b \lambda} + c, \tag{1}
$$
Where $a$, $b$, and $c$ are constants and $\lambda$ is the growth rate.

In [44]:
# Define an exponential function to fit the cell volume
def exp_fit(lam, a, b, c):
    return a * np.exp(lam * b) + c

lam_range = np.linspace(0, 2.5, 100)
fit_dfs = []
for g, d in vol_data.groupby(['strain']):
    # Perform the fit
    popt = scipy.optimize.curve_fit(exp_fit,
                                d['growth_rate_hr'].values, 
                                d['cell_volume_fL'].values)

    # Unpack and print the parameters
    a, b, c = popt[0]
    print(f"""
    Best-fit parameters for {g} cell volume
    ============================================
    a = {a:0.3f} µm^3
    b = {b:0.3f} hr
    c = {c:0.3f} µm^3
    """)

    # Compute the fit and create a dataframe
    fit_df = pd.DataFrame([])
    fit_df['growth_rate_hr'] = lam_range
    fit_df['cell_volume_fL'] = a * np.exp(b * lam_range) + c
    fit_df['strain'] = g
    fit_dfs.append(fit_df)
fit_df = pd.concat(fit_dfs, sort=False)


    Best-fit parameters for MG1655 cell volume
    a = 0.816 µm^3
    b = 0.869 hr
    c = -0.404 µm^3
    

    Best-fit parameters for NCM3722 cell volume
    a = 0.236 µm^3
    b = 1.338 hr
    c = 0.023 µm^3
    


Let's see how good this fit is by plotting it. 

In [45]:
data_plot = alt.Chart(
            data =vol_data
            ).mark_point(
                size=100, 
                opacity=0.75
            ).encode(
                x='growth_rate_hr:Q',
                y='cell_volume_fL:Q',
                color='strain:N'
            )

fit_plot = alt.Chart(
            data=fit_df
            ).mark_line( 
                size=1
            ).encode(
                x='growth_rate_hr:Q',
                y='cell_volume_fL:Q',
                color='strain:N'
            )

data_plot + fit_plot

alt.LayerChart(...)

This seems to do a pretty good job at an empirical level. It's again important to note that there is a big difference between NCM3722 and MG1655.
We should do the same procedure to get an empirical description of how the cell length and width scale with growth rate as we will use 
these quantities to compute the envelope density. 

In [66]:
# Perform a similar fitting procedure for cell width and length
dim_dfs = []
for g, d in vol_data.groupby(['strain']):

    # Do the fits
    length_popt = scipy.optimize.curve_fit(exp_fit, 
                                          d['growth_rate_hr'].values, 
                                          d['cell_length_um'].values)
    width_popt = scipy.optimize.curve_fit(exp_fit, 
                                          d['growth_rate_hr'].values, 
                                          d['cell_width_um'].values,
                                          maxfev=100000)
    # unpack and compute the fits
    length_a, length_b, length_c = length_popt[0]
    length_fit = length_a * np.exp(length_b * lam_range) + length_c
    width_a, width_b, width_c = width_popt[0]
    width_fit = width_a * np.exp(width_b * lam_range) + width_c

    # Assemble the dataframe
    _df = pd.DataFrame([])
    _df['growth_rate_hr'] = lam_range
    _df['cell_length_um'] = length_fit
    _df['cell_width_um'] = width_fit
    _df['strain'] = g
    dim_dfs.append(_df)

    # Print out the results
    print(f"""
    Cell Dimension fits for {g} 
    ==================================

    Width 
    ------
    a = {width_a :0.3f} µm
    b = {width_b: 0.4f} hr
    c = {width_c:0.3f} µm

    Length 
    ------
    a = {length_a:0.3f} µm
    b = {length_b:0.3f}  hr
    c = {length_c:0.3f} µm
    """)

dim_df = pd.concat(dim_dfs)


    Cell Dimension fits for MG1655 

    Width 
    ------
    a = 642.068 µm
    b =  0.0003 hr
    c = -641.434 µm

    Length 
    ------
    a = 1.036 µm
    b = 0.798  hr
    c = 0.938 µm
    

    Cell Dimension fits for NCM3722 

    Width 
    ------
    a = 0.394 µm
    b =  0.4738 hr
    c = 0.061 µm

    Length 
    ------
    a = 0.470 µm
    b = 0.852  hr
    c = 1.642 µm
    


In [65]:
# Generate plots to see how it looks
vol_data['type'] = 'experiment'
dim_df['type'] = 'fit'

charts = []
for dim in ['cell_length_um', 'cell_width_um']:
    dim_point = alt.Chart(
            data=vol_data
            ).mark_point(
                size=100,
                opacity=0.75
            ).encode(
                x='growth_rate_hr:Q',
                y=dim + ':Q',
                color='strain:N')
    dim_fit = alt.Chart(
            data=dim_df,
            ).mark_line(
                size=1
            ).encode(
                x='growth_rate_hr:Q',
                y=dim + ':Q',
                color='strain:N'
            )
    charts.append(dim_point + dim_fit)

alt.hconcat(charts[0],charts[1])

alt.HConcatChart(...)

These fits also look okay, though there was some trouble fitting the width from MG1655. Nevertheless, to use these we can set up functions to compute the dimensions froma supplied growth rate. 

In [74]:
def empirical_cell_volume(lam, strain='MG1655'):
    """
    Computes the cell volume at a given growth rate. 

    Parameters 
    -----------
    lam : positive float
        The growth rate in units of inverse hours. 
    strain : string 'NCM3722' or 'MG1655'
        The strain parameters to use. Only NCM3722 or MG1655 can be supplied

    Returns
    -------
    vol : positive float
        The cell volume in units of fL (same as cubic microns)
    """

    if strain == 'MG1655':
        a = 0.816
        b = 0.869
        c = -0.404
    elif strain == 'NCM3722':
        a = 0.236
        b = 1.338
        c = 0.023
    vol = a * np.exp(b * lam) + c
    return vol

def empirical_cell_length(lam, strain='MG1655'):
    """ 
    Computes the cell length at a given growth rate. 

    Parameters 
    -----------
    lam : positive float
        The growth rate in units of inverse hours. 
    strain : string 'NCM3722' or 'MG1655'
        The strain parameters to use. Only NCM3722 or MG1655 can be supplied

    Returns
    -------
    length : positive float
        The cell length in units of micron
    """

    if strain == 'MG1655':
        a = 1.036
        b = 0.798
        c = 0.938
    elif strain == 'NCM3722':
        a = 0.470
        b = 0.852
        c = 1.642
    length = a * np.exp(b * lam) + c
    return length


def empirical_cell_width(lam, strain='MG1655'):
    """ 
    Computes the cell width at a given growth rate. 

    Parameters 
    -----------
    lam : positive float
        The growth rate in units of inverse hours. 
    strain : string 'NCM3722' or 'MG1655'
        The strain parameters to use. Only NCM3722 or MG1655 can be supplied

    Returns
    -------
    width : positive float
        The cell width in units of micron
    """

    if strain == 'MG1655':
        a = 642.068 
        b = 0.003 
        c = -641.434
    elif strain == 'NCM3722':
        a = 0.394
        b = 0.474
        c = 0.061 
    length = a * np.exp(b * lam) + c
    return length


## Estimation of Protein Density Scaling

With functions in hand that do a pretty good job of empirically determining the cell length, width, and volume as a function of growth rate, we now need an estimate of what the protein density per unit cell volume. As mentioned earlier, Basan *et al.* did a careful measurement of the amount of protein biomass per OD ml and the number of cells per OD ml, as shown below.

In [70]:
# Load the calibration data
cal_data = pd.read_csv('../../data/source/Basan2015/Basan2015_calibration_curve.csv')
cal_data['cells_per_OD_1E8'] = cal_data['cells_per_OD'].values / 1E8
# Plot the protein biomass and cell number as a function of growth rate. 
cal_plot = alt.Chart(
            data=cal_data,
            ).mark_point(
                size=100,
                opacity=0.75
            ).encode(
                x='growth_rate_hr:Q',
                y=alt.Y(alt.repeat('column'), type='quantitative')
            ).repeat(
                column=['ug_protein_per_OD', 'cells_per_OD_1E8']
            )
cal_plot

alt.RepeatChart(...)

These data make intuitive sense -- in general, the protein density you get per OD is roughly constant whereas the total number of cells per OD unit decreases as growth rate increases (meaning as cells get larger). We can compute the total mass of a cell as a function of growth rate as the ratio of these two quantities.

In [72]:
cal_data['fg_protein_per_cell'] = 1E9 * cal_data['ug_protein_per_OD'].values / cal_data['cells_per_OD'].values
cal_plot = alt.Chart(
            data=cal_data
            ).mark_point(
                size=100,
                opacity=0.75
            ).encode(
                x='growth_rate_hr:Q',
                y='fg_protein_per_cell:Q'
            )
cal_plot

alt.Chart(...)

For our case, we need to be able to compute the protein density *per unit of cell volume*, not per cell. While Basan *et. al.* report their cell volumes, they seem to be consistenly larger than expected. We can use our empirical descriptions of cell volume as a function of growth rate to get an estimate of the "correct" volume at each growth rate. 

In [75]:
# Calculate the corrected cell volume for NCM
cal_data['corrected_volume_fL'] = empirical_cell_volume(cal_data['growth_rate_hr'], strain='NCM3722')
cal_plot = alt.Chart(
            data=cal_data
            ).mark_point(
                size=100,
                opacity=0.75
            ).encode(
                x='corrected_volume_fL:Q',
                y='fg_protein_per_cell:Q'
            )
cal_plot

alt.Chart(...)

This looks like an approximately linear relationship between cell size and cell volume, which is would be as expected if the density was approximately constant. To that end, we can 
do a simple linear regression to get the slope and intercept, and therefore reliably compute the protein mass per cell as a function of cell volume. 

In [80]:
# Perform a simple linear regression
popt = scipy.stats.linregress(cal_data['corrected_volume_fL'], cal_data['fg_protein_per_cell'])

# Unpack parameters report the results
density  = popt[0]
offset = popt[1]

print(f"""
Estimated protein density from Basan et al. 2015
=================================================
density, ρ = {density:0.3f} fg protein / fL
offset, c = {offset:0.3f} fg protein / cell
""")

# Compute the fits for display
vol_range = np.linspace(0, 3.5, 100)
fit = density * vol_range + offset
fit_df = pd.DataFrame([])
fit_df['volume'] = vol_range
fit_df['protein_per_cell'] = fit

# Set the canvases for plotting
cal_points = alt.Chart(
        data=cal_data
        ).mark_point(
            size=100,
            opacity=0.75
        ).encode(
            x=alt.X('corrected_volume_fL:Q', title='cell volume [fL]'),
            y=alt.Y('fg_protein_per_cell:Q', title='fg protein / fL'),
        )   

cal_fit = alt.Chart(
        data=fit_df
        ).mark_line(
            size=1
        ).encode(
            x=alt.X('volume:Q', title='cell volume [fL]'),
            y=alt.Y('protein_per_cell:Q', title='fg protein / fL')
        )
cal_points + cal_fit


Estimated protein density from Basan et al. 2015
density, ρ = 320.843 fg protein / fL
offset, c = 34.662 fg protein / cell



alt.LayerChart(...)

This also looks reasonable, though more data would always be better for getting a sense of uncertainty. Assuming that the protein density is similar between NCM3722 and MG1655 (which I think is reasonable), we can define a function that computes the fg protein per cell as a function of the cell volume, which we can compute as a function of the growth rate!

In [81]:
def compute_protein_per_cell(lam, strain='MG1655'):
    """
    Given calibration curve and empirical relations, compute the total mas of
    of protein per cell as a function of the growth rate. 

    Parameters 
    -----------
    lam : positive float 
        The growth rate (in inverse hours) at which you want to compute the 
        protein mass 
    strain : string, 'NCM3722' or 'MG1655'
        The E. coli strain used in determination of cell volume. Default is
        MG1655. 

    Returns
    --------
    prot : positive float
        The total mass of protein per cell in units of femtograms. 
    """

    # Determine the cell volume
    vol = empirical_cell_volume(lam, strain)

    # Compute and return the protein 
    prot = 34.662 + 320.843 * vol
    return prot

## Converting From Mass Fraction to Total Mass
To make use of the proteomic data, we have to be able to compute the mass of each protein and/or protein class. As of now, 

In [5]:
# Load the proteomics data 
data = pd.read_csv('../../data/compiled_mass_fractions.csv')
data = data[~data['go_terms'].isnull()]

# Thickness of the periplasm
delta = 0.025

# Compute the cell volume from the growth rate
data['cell_width'] = size.empirical.lambda2width(data['growth_rate_hr'])
data['cell_length'] = size.empirical.lambda2length(data['growth_rate_hr'])
data['cell_volume'] = size.analytical.volume(data['cell_length'], data['cell_width'])
data['envelope_volume'] = size.analytical.envelope_volume(data['cell_length'], 
                                                          data['cell_width'], 
                                                          delta)

# Given the slope, compute the total protein per cell
tot_mass = popt[0] * data['cell_volume'] + popt[1]
data['fg_per_cell'] = data['mass_frac'] * tot_mass


# Using the GO classification, compute the mass, mass fraction, and periplasmic protein density
periplasm = data[data['go_terms'].str.contains('GO:0042597')]
periplasm_grouped = periplasm.groupby(['dataset_name', 'condition', 
                                       'growth_rate_hr', 'cell_volume',
                                       'envelope_volume']).sum().reset_index()
periplasm_grouped['density'] = periplasm_grouped['fg_per_cell'].values / periplasm_grouped['envelope_volume']